# Day 7: Amplification Circuit
https://adventofcode.com/2019/day/7

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools

Instruction set and intcode interpreter V3 from Part 1 (days 1-4)

In [ ]:
class IntcodeParameter():
    value = None
    mode = None
    
    def __init__(self, value, mode):
        self.value = value
        self.mode = mode
#         assert( mode in [0, 1], 'UNKNOWN PARAMETER MODE {}'.format(mode))
        assert mode in [0, 1]

    def checkBoundedParamter(self, program):
        if self.mode == 0:
            assert( self.value >= 0 and self.value < len(program))
        return True
        
    def evaluateParameter(self, program):
        if self.mode == 0:
            # position mode
            return program[self.value]
        elif self.mode == 1:
            # immediate mode
            return self.value
        else:
            raise Exception('UNKNOWN PARAMETER MODE {}. Value {}. Program {}'.format(self.mode, self.value, program))
    
    def toString(self, program):
        return '[value {}, mode {}, evaluation {}]'.format(self.value, self.mode, self.evaluateParameter(program))

In [ ]:
class IntcodeInstruction():
    endComputations = False
    parameters = []
    
    def mustEndComputations(self):
        return self.endComputations
    
    def getInstructionSize(self):
        return len(self.parameters) + 1 #parameters + opcode
    
    def getNextInstructionPointer(self, program, IP):
        newIP = IP + self.getInstructionSize()
        assert( newIP >= 0 and newIP < len(program))
        return newIP
    
    def extractParameters(self, program, IP):
        pass
    
    def performComputation(self, program):
        pass
    
    def checkParameters(self, program):
        for param in self.parameters:
            param.checkBoundedParamter(program)
    
    def loadParameters(self, program, IP):
        self.parameters = self.extractParameters(program, IP)
        self.checkParameters(program)
    
    def compute(self, program, IP):
        self.performComputation(program)
    
    def toString(self, program):
        msg = self.__class__.__name__
        for param in self.parameters:
            msg += param.toString(program)
        
        return (msg)

class OneParameterInst(IntcodeInstruction):
    def extractParameters(self, program, IP):
        parameters = []
        modes = program[IP] // 100
        parameters.append(IntcodeParameter(program[IP + 1], modes % 10))
        
        return parameters

class TwoParametersInst(OneParameterInst):
    def extractParameters(self, program, IP):
        parameters = super().extractParameters(program, IP)
        modes = program[IP] // 100
        #1 parameters + 1 result
        modes //= 10
        parameters.append(IntcodeParameter(program[IP + 2], modes % 10))
        
        return parameters
    
class ThreeParameterInst(TwoParametersInst):
    def extractParameters(self, program, IP):
        parameters = super().extractParameters(program, IP)
        modes = program[IP] // 100
        #2 parameters + 1 result
        modes //= 10
        modes //= 10
        parameters.append(IntcodeParameter(program[IP + 3], modes % 10))
        
        return parameters
    
class AdditionInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) + self.parameters[1].evaluateParameter(program)
        program[self.parameters[2].value] = resultValue
        
class MultiplyInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = self.parameters[0].evaluateParameter(program) * self.parameters[1].evaluateParameter(program)
        program[self.parameters[2].value] = resultValue

class LessThanInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) < self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program[self.parameters[2].value] = resultValue
        
class EqualsInst(ThreeParameterInst):
    def performComputation(self, program):
        resultValue = 0
        if self.parameters[0].evaluateParameter(program) == self.parameters[1].evaluateParameter(program):
            resultValue = 1
        program[self.parameters[2].value] = resultValue
        
        
class InputInst(OneParameterInst):
    def performComputation(self, program):
        n = input('Solicitando dato de entrada:')
        
        # TODO: Validar dato!!! Debe ser entero, dado que luego podrá ser
        # usado para operar con él o ser un opcode, etc..
        
        program[self.parameters[0].value] = int(n)

class OutputInst(OneParameterInst):
    def performComputation(self, program):
        print('>>>OUTPUT: {}'.format(self.parameters[0].evaluateParameter(program)))

class JumpIfTrueInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfTrueInst!')
        pass

    def getNextInstructionPointer(self, program, IP):
        if self.parameters[0].evaluateParameter(program) != 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program, IP)

class JumpIfFalseInst(TwoParametersInst):
    def performComputation(self, program):
#         print('JumpIfFalseInst!')
        pass

    def getNextInstructionPointer(self, program, IP):
        if self.parameters[0].evaluateParameter(program) == 0:
            return self.parameters[1].evaluateParameter(program)
        
        return super().getNextInstructionPointer(program, IP)
        
class QuitInst(IntcodeInstruction):
    def __init__(self):
        self.endComputations = True

    def extractParameters(self, program, IP):
        return []
        
    def performComputation(self, program):
        pass

In [ ]:
class IntcodeInterpreterV3():
    
    IP = None
    instructions = {}
    
    def __init__(self):
        self.IP = 0
        # TODO: Inject instruction set
        self.instructions[1]  = AdditionInst
        self.instructions[2]  = MultiplyInst
        self.instructions[99] = QuitInst
        self.instructions[3]  = InputInst
        self.instructions[4]  = OutputInst
        self.instructions[7]  = LessThanInst
        self.instructions[8]  = EqualsInst
        self.instructions[5]  = JumpIfTrueInst
        self.instructions[6]  = JumpIfFalseInst

    def getInstruction(self, program, opcode):
        opcode = opcode % 100
        if opcode in self.instructions:
            instruction = self.instructions[opcode]()
            instruction.loadParameters(program, self.IP)
            return instruction
        return None
        
    def compute(self, program):
#         print(f'Executing {program}!')
        self.IP = 0
        while True:
            opcode = program[self.IP]
            # Extract next instruction            
            instruction = self.getInstruction(program, opcode)
            
            if instruction is None:
                raise Exception('UNKNOWN opcode {} ({}) at position {}.\nPROGRAM: {}'.format(opcode, program[self.IP], self.IP, program))
                
            # Exit if it must
            if instruction.mustEndComputations():
                break
            else:
                # Otherwise, compute instruction
                instruction.compute(program, self.IP)
                # Change IP (Instruction Pointer)
                self.IP = instruction.getNextInstructionPointer(program, self.IP)
            
#             print('DEBUGGGGGG')
#             print(program)
#             print('IP {}'.format(self.IP))
#             print('DEBUGGGGGG')
            
        return program

InputFromProgramInst will replace Original InputInst (opcode 3) and OutputToProgramInst will replace OutputInst (opcode 4). This implementation uses the very program as a 'device' to read and write values.

* InputFromProgramInst will have an inner pointer and will take the value from the program at the position this inner pointer points to.
* OutputToProgramInst will write the value to the last position of the program

It would be better:
* To implement a Device class with two operations: read and write
* Derive OneParameterIOInst from OneParameterInst, to implement instructions that will operate with devices.
* Implement InputFromDeviceInst and OutputToDeviceInt, that reads values and writes values to a device.

Some programs won't compute the right answer if we mangle the program because if they use negative indexes to access parameters they won't be at the expected position.

In [ ]:
class InputFromProgramInst(OneParameterInst):
    pointer = None
   
    def resetPointer(program):
        InputFromProgramInst.pointer = len(program) - 3
    
    def performComputation(self, program):
        if InputFromProgramInst.pointer >= len(program):
            raise Exception('memory corruption!!!')
            
        n = program[InputFromProgramInst.pointer]
#         print('InputFromProgramInst accedo a posicion {} (Value: {})'.format(InputFromProgramInst.pointer, n))
        
        InputFromProgramInst.pointer += 1
        
        # TODO: Validar dato!!! Debe ser entero, dado que luego podrá ser
        # usado para operar con él o ser un opcode, etc..
        
        program[self.parameters[0].value] = int(n)

class OutputToProgramInst(OneParameterInst):
    def performComputation(self, program):
#         print('>>>OUTPUT: {}'.format(self.parameters[0].evaluateParameter(program)))
        n = self.parameters[0].evaluateParameter(program)
        if n == None:
            raise Exception('Trying to output unknown value!!')

#         print('OutputToProgramInst accedo a posicion {} (Value: {})'.format(len(program) - 1, n))
        program[len(program) - 1] = n

In [ ]:
class Amplifier():
    amplifierCode = None
    machine = None
    program = None
    input_value = None
    output_value = None
    
    def __init__(self, code, machine):
        self.amplifierCode = code
        self.machine = machine
        self.machine.instructions[3]  = InputFromProgramInst
        self.machine.instructions[4]  = OutputToProgramInst
        
    def loadProgram(self, program):
        # adding a staging zone to the program
        # 0 position = 99, to terminate original program in case it has no end
        # 1 position = phase
        # 2 position = initial value
        # 3 position = output value
        self.program = program + [99, None, None, None]
    
    def amplifySignal(self, phase, input_value):

        # reset state
        self.machine.instructions[3].resetPointer(self.program)
        execution_program = self.program.copy()
        # setting parameters
        execution_program[len(self.program) - 3] = phase
        execution_program[len(self.program) - 2] = input_value
        
        # execute program
        self.machine.compute(execution_program)
        
        # gather result
        output_value = execution_program[len(self.program) - 1]

        print(f'\tAmplifier {self.amplifierCode}. Phase {phase} input {input_value}. output {output_value}')
        
        return output_value
        

### Tests

In [ ]:
def performPermutation(amplifiers, permutation):
    input_signal = 0
    for ampID in range(len(amplifiers)):
        amp = amplifiers[ampID]
        phase = permutation[ampID]
        output_signal = amp.amplifySignal(phase, input_signal)
        input_signal = output_signal
    return output_signal

In [ ]:
def getAmplifiers(program):
    amplifiers = [Amplifier(code, IntcodeInterpreterV3()) for code in ['A', 'B', 'C', 'D', 'E']]
    for amp in amplifiers:
        amp.loadProgram(program)

    return amplifiers

In [ ]:
def getMaxSignal(program):
    amplifiers = getAmplifiers(program)
    
    # calculate permutations
    permutations = list(itertools.permutations([0, 1, 2, 3, 4]))
    
#     itermax = 500
#     iteration = 0
    
    bestPermutation = None
    maxSignal = None
    for permutation in permutations:
        output_signal = performPermutation(amplifiers, permutation)
        print('Permutation {} yields {}'.format(permutation, output_signal))
        
        if output_signal == None:
            continue
        
        if bestPermutation == None:
            bestPermutation = permutation
            maxSignal = output_signal
            print(f'New maxSignal {output_signal} using permutation {permutation}')
        else:
            if output_signal > maxSignal:
                maxSignal = output_signal
                bestPermutation = permutation
                print(f'New maxSignal {output_signal} using permutation {permutation}')
        
#         iteration += 1
#         if iteration > 5:
#             break
    return bestPermutation, maxSignal

#### Test 1
 * 3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
 * Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0)

In [ ]:
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

amplifiers = getAmplifiers(program.copy())


print('Testing exact permutation')
permutation = [4, 3, 2, 1, 0]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy())
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

#### Test 2
* 3,23,3,24,1002,24,10,24,1002,23,-1,23, 101,5,23,23,1,24,23,23,4,23,99,0,0
* Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4)

In [ ]:
program = [3,23,3,24,1002,24,10,24,1002,23,-1,23, 101,5,23,23,1,24,23,23,4,23,99,0,0]

amplifiers = getAmplifiers(program)

print('Testing exact permutation')
permutation = [0, 1, 2, 3, 4]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy())
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

#### Test 3
* 3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
* Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2)

In [ ]:
program = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]

amplifiers = getAmplifiers(program)

print('Testing exact permutation')
permutation = [1,0,4,3,2]
result = performPermutation(amplifiers, permutation)
print( f'Resultado es {result}')
print(50 * '-')

print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(program.copy())
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

### Solution

In [ ]:
input_7 = r'data\aoc2019-input-day7.txt'
with open(input_7, 'r') as f:
    data7 = [int(data) for data in f.read().split(',') if len(data) > 0]

In [ ]:
print('Obtaining max signal and permutation')
bestPerm, maxsignal = getMaxSignal(data7.copy())
print( f'Maxsignal {maxsignal} using permutation {bestPerm}')

>>>OUTPUT: 17406

## Part 2